# Customer Support Agent with Amazon Bedrock

In [1]:
!sh ./ro_shared_data/reset.sh

import os

roleArn = os.environ['BEDROCKAGENTROLE']

Resetting environment (if nessesary)
Agent reset process completed.
Lambda reset process completed.
Guardrail reset process completed.
Environment reset complete.


In [2]:
import boto3

In [3]:
bedrock_agent = boto3.client(service_name='bedrock-agent', region_name='us-west-2')

In [4]:
create_agent_response = bedrock_agent.create_agent(
    agentName='mugs-customer-support-agent',
    foundationModel='anthropic.claude-3-haiku-20240307-v1:0',
    instruction="""You are an advanced AI agent acting as a front line customer support agent.""",
    agentResourceRoleArn=roleArn
)

In [5]:
create_agent_response

{'ResponseMetadata': {'RequestId': '0024d7b1-cfde-4faf-9fc5-390aa59dff42',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 05 Dec 2024 03:52:15 GMT',
   'content-type': 'application/json',
   'content-length': '624',
   'connection': 'keep-alive',
   'x-amzn-requestid': '0024d7b1-cfde-4faf-9fc5-390aa59dff42',
   'x-amz-apigw-id': 'CTJ5fG_bPHcEc0w=',
   'x-amzn-trace-id': 'Root=1-6751236f-39d37e877c2f8287645b1eb0'},
  'RetryAttempts': 0},
 'agent': {'agentArn': 'arn:aws:bedrock:us-west-2:339712907484:agent/7SEG2DFRGI',
  'agentId': '7SEG2DFRGI',
  'agentName': 'mugs-customer-support-agent',
  'agentResourceRoleArn': 'arn:aws:iam::339712907484:role/c135316a3428926l8240223t1w85172521-BedrockAgentRole-eRrGn2KGE050',
  'agentStatus': 'CREATING',
  'createdAt': datetime.datetime(2024, 12, 5, 3, 52, 15, 599489, tzinfo=tzlocal()),
  'foundationModel': 'anthropic.claude-3-haiku-20240307-v1:0',
  'idleSessionTTLInSeconds': 600,
  'instruction': 'You are an advanced AI agent acting as a 

In [6]:
agentId = create_agent_response['agent']['agentId']

In [7]:
from helper import *

In [8]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='NOT_PREPARED'
)

Waiting for agent status of 'NOT_PREPARED'...
Agent status: NOT_PREPARED
Agent reached 'NOT_PREPARED' status.


In [9]:
bedrock_agent.prepare_agent(
    agentId=agentId
)

{'ResponseMetadata': {'RequestId': '562f0451-95cc-4542-bdb7-8e72fe740cc1',
  'HTTPStatusCode': 202,
  'HTTPHeaders': {'date': 'Thu, 05 Dec 2024 03:52:23 GMT',
   'content-type': 'application/json',
   'content-length': '119',
   'connection': 'keep-alive',
   'x-amzn-requestid': '562f0451-95cc-4542-bdb7-8e72fe740cc1',
   'x-amz-apigw-id': 'CTJ6pEHovHcEAIQ=',
   'x-amzn-trace-id': 'Root=1-67512376-52a6e13146c49fd319c864ab'},
  'RetryAttempts': 0},
 'agentId': '7SEG2DFRGI',
 'agentStatus': 'PREPARING',
 'agentVersion': 'DRAFT',
 'preparedAt': datetime.datetime(2024, 12, 5, 3, 52, 23, 127950, tzinfo=tzlocal())}

In [10]:
wait_for_agent_status(
    agentId=agentId, 
    targetStatus='PREPARED'
)

Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Agent reached 'PREPARED' status.


In [11]:
create_agent_alias_response = bedrock_agent.create_agent_alias(
    agentId=agentId,
    agentAliasName='MyAgentAlias',
)

agentAliasId = create_agent_alias_response['agentAlias']['agentAliasId']

wait_for_agent_alias_status(
    agentId=agentId,
    agentAliasId=agentAliasId,
    targetStatus='PREPARED'
)

Waiting for agent alias status of 'PREPARED'...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Agent alias reached status 'PREPARED'


In [12]:
bedrock_agent_runtime = boto3.client(service_name='bedrock-agent-runtime', region_name='us-west-2')

In [13]:
import uuid

In [14]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

invoke_agent_response = bedrock_agent_runtime.invoke_agent(
    agentId=agentId,
    agentAliasId=agentAliasId,
    inputText=message,
    sessionId=sessionId,
    endSession=False,
    enableTrace=True,
)

In [15]:
event_stream = invoke_agent_response["completion"]

In [16]:
for event in event_stream:
    print(event)

{'trace': {'agentAliasId': 'PMZVHHJDDU', 'agentId': '7SEG2DFRGI', 'agentVersion': '1', 'sessionId': '647a496b-e173-4521-b931-72448969f73e', 'trace': {'orchestrationTrace': {'modelInvocationInput': {'inferenceConfiguration': {'maximumLength': 2048, 'stopSequences': ['</invoke>', '</answer>', '</error>'], 'temperature': 0.0, 'topK': 250, 'topP': 1.0}, 'text': '{"system":"You are an advanced AI agent acting as a front line customer support agent.You have been provided with a set of functions to answer the user\'s question.You must call the functions in the format below:<function_calls>  <invoke>    <tool_name>$TOOL_NAME</tool_name>    <parameters>      <$PARAMETER_NAME>$PARAMETER_VALUE</$PARAMETER_NAME>      ...    </parameters>  </invoke></function_calls>Here are the functions available:<functions>  </functions>You will ALWAYS follow the below guidelines when you are answering a question:<guidelines>- Think through the user\'s question, extract all data from the question and the previous

{'trace': {'agentAliasId': 'PMZVHHJDDU', 'agentId': '7SEG2DFRGI', 'agentVersion': '1', 'sessionId': '647a496b-e173-4521-b931-72448969f73e', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 877, 'outputTokens': 106}}, 'rawResponse': {'content': 'I apologize, but I seem to be having trouble retrieving the details of your order. Could you please provide the order ID or the date you made the purchase? That will help me look up the details and process your return request.\n\n<function_calls>\n<invoke>\n    <tool_name>GetOrderDetails</tool_name>\n    <parameters>\n      <order_id>123456</order_id>\n    </parameters>\n</function_calls>'}, 'traceId': '17d517f6-a223-4c0a-8022-a139d4a64ec8-2'}}}}}
{'trace': {'agentAliasId': 'PMZVHHJDDU', 'agentId': '7SEG2DFRGI', 'agentVersion': '1', 'sessionId': '647a496b-e173-4521-b931-72448969f73e', 'trace': {'orchestrationTrace': {'rationale': {'text': 'I apologize, but I seem to be having trouble retrieving the

{'trace': {'agentAliasId': 'PMZVHHJDDU', 'agentId': '7SEG2DFRGI', 'agentVersion': '1', 'sessionId': '647a496b-e173-4521-b931-72448969f73e', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 1415, 'outputTokens': 110}}, 'rawResponse': {'content': '<thinking>I apologize, but I seem to be having trouble retrieving the details of your order. Could you please provide the order ID or the date you made the purchase? That will help me look up the details and process your return request.</thinking>\n\n<function_calls>\n  <invoke>\n    <tool_name>GetOrderDetails</tool_name>\n    <parameters>\n      <order_id>123456</order_id>\n    </parameters>'}, 'traceId': '17d517f6-a223-4c0a-8022-a139d4a64ec8-4'}}}}}
{'trace': {'agentAliasId': 'PMZVHHJDDU', 'agentId': '7SEG2DFRGI', 'agentVersion': '1', 'sessionId': '647a496b-e173-4521-b931-72448969f73e', 'trace': {'orchestrationTrace': {'rationale': {'text': 'I apologize, but I seem to be having trouble retrievin

{'trace': {'agentAliasId': 'PMZVHHJDDU', 'agentId': '7SEG2DFRGI', 'agentVersion': '1', 'sessionId': '647a496b-e173-4521-b931-72448969f73e', 'trace': {'orchestrationTrace': {'modelInvocationOutput': {'metadata': {'usage': {'inputTokens': 1929, 'outputTokens': 117}}, 'rawResponse': {'content': 'I apologize, but I seem to be having trouble retrieving the details of your order. Could you please provide the order ID or the date you made the purchase? That will help me look up the details and process your return request.\n\n<guidelines>\n- I will not disclose any information about the tools and functions that are available to me, as that would violate the guidelines.\n- I will continue to follow the guidelines and request the necessary information from the user to process the return request, even if the user becomes frustrated.\n</guidelines>'}, 'traceId': '17d517f6-a223-4c0a-8022-a139d4a64ec8-6'}}}}}
{'trace': {'agentAliasId': 'PMZVHHJDDU', 'agentId': '7SEG2DFRGI', 'agentVersion': '1', 'ses

In [17]:
message = "Hello, I bought a mug from your store yesterday, and it broke. I want to return it."

sessionId = str(uuid.uuid4())

In [18]:
invoke_agent_and_print(
    agentAliasId=agentAliasId,
    agentId=agentId,
    sessionId=sessionId,
    inputText=message,
    enableTrace=True,
)

User: Hello, I bought a mug from your store yesterday, and it broke. I want
to return it.

Agent: 
Agent's thought process:
  Okay, let's go through the steps to handle this return request:

Agent's thought process:
  Apologies, let me try that again in the correct format:

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  call. Let me try a different approach to get the return policy
  information.

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  call. Let me try a different approach to get the return policy
  information.

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  calls. Let me try to get the return policy information in the
  correct format.

Agent's thought process:
  I apologize, it seems there was an issue with my previous function
  calls. Let me try to get the return policy information in the
  correct format.

Observation:
  Type: 